# Reading Archaic SSW `.genx` Files
Tests for reading `.genx` files better so that they are easier to work with when we are calculating response functions.

In [3]:
from scipy.io import readsav
from astropy.table import Table,QTable
import astropy.units as u
import numpy as np
import os

In [190]:
def genx_to_table(input_dir,channel_list,version):
    properties = [('wave','wavelength'),('wavemin','minimum_wavelength'),('wavestep','wavelength_interval'),
                  ('wavenumsteps','number_wavelength_intervals'),('effarea','effective_area'),('geoarea','geometric_area_ccd'),
                  ('platescale','plate_scale'),('elecperdn','electron_per_dn'),('elecperev','electron_per_ev'),
                  ('fp_filter','focal_plane_filter_efficiency'),('ent_filter','entire_filter_efficiency'),
                  ('primary','primary_mirror_reflectance'),('secondary','secondary_mirror_reflectance'),
                  ('ccd','quantum_efficiency_ccd'),('contam','ccd_contamination')
                 ]
    unitless = u.m/u.m
    units = [u.angstrom,u.angstrom,u.angstrom,unitless,u.cm**2,u.cm**2,1/u.cm,u.electron/u.count,u.electron/u.eV,
             unitless,unitless,unitless,unitless,unitless,unitless]
    units = {p[1] : u for p,u in zip(properties,units)}
    units['channel'] = u.angstrom
    
    instrument_files = [os.path.join(input_dir,'aia_V{0}_all_fullinst'.format(version)),
                        os.path.join(input_dir,'aia_V{0}_fuv_fullinst'.format(version))
                       ]
    field_name = 'A{0}_FULL'
    
    #read in values
    rows = []
    for instr_file in instrument_files:
        instrument_data = readsav(instr_file)['data']
        for channel in channel_list:
            if field_name.format(channel).upper() not in instrument_data.dtype.names:
                continue
            row = {'channel':channel}
            channel_data = instrument_data[field_name.format(channel)][0]
            for prop in properties:
                if prop[0].upper() not in channel_data.dtype.names:
                    print('Cannot find {0} for channel {1} in file {2}'.format(prop[0],channel,instr_file))
                    print('Setting {} to 1'.format(prop[1]))
                    row[prop[1]] = 1
                else:
                    row[prop[1]] = channel_data[prop[0]][0]
            rows.append(row)
    
    #assign units
    table = QTable(rows=rows,names=tuple(['channel']+[p[1] for p in properties]))
    for name in table.colnames:
        try:
            table[name].unit = units[name]
        except TypeError:
            pass
        
    return table

Now, test out the function.

In [191]:
channel_list = [94,335,171,193,211,131,304,1600,1700,4500]
genx_dir = '/Users/willbarnes/Documents/Projects/gsoc/aia_response/ssw_aia_response_data/'
version = 6

In [192]:
table = genx_to_table(genx_dir,channel_list,version)

Cannot find contam for channel 1600 in file /Users/willbarnes/Documents/Projects/gsoc/aia_response/ssw_aia_response_data/aia_V6_fuv_fullinst
Setting ccd_contamination to 1
Cannot find contam for channel 1700 in file /Users/willbarnes/Documents/Projects/gsoc/aia_response/ssw_aia_response_data/aia_V6_fuv_fullinst
Setting ccd_contamination to 1
Cannot find contam for channel 4500 in file /Users/willbarnes/Documents/Projects/gsoc/aia_response/ssw_aia_response_data/aia_V6_fuv_fullinst
Setting ccd_contamination to 1


In [193]:
table

channel,wavelength,minimum_wavelength,wavelength_interval,number_wavelength_intervals,effective_area,geometric_area_ccd,plate_scale,electron_per_dn,electron_per_ev,focal_plane_filter_efficiency,entire_filter_efficiency,primary_mirror_reflectance,secondary_mirror_reflectance,quantum_efficiency_ccd,ccd_contamination
Angstrom,Angstrom,Angstrom,Angstrom,,cm2,cm2,1 / cm,electron / ct,electron / eV,,,,,,
float64,object,float32,float32,int32,object,float32,float32,float32,float32,object,object,object,object,object,object
94.0,"[ 25. 25.10000038 25.20000076 ..., 899.79998779 899.90002441 900. ]",25.0,0.1,8751,"[ 5.27489294e-12 5.14545784e-12 5.01929488e-12 ..., 2.45101570e-14 2.45180802e-14 2.45260016e-14]",83.0,8.46158e-12,18.3,0.273973,"[ 5.17922007e-02 5.09420000e-02 5.01057990e-02 ..., 1.00001000e-06 1.00001000e-06 1.00001000e-06]","[ 5.17922007e-02 5.09420000e-02 5.01057990e-02 ..., 1.00001000e-06 1.00001000e-06 1.00001000e-06]","[ 5.36785637e-06 5.39225630e-06 5.41665577e-06 ..., 8.52360651e-02 8.52409825e-02 8.52458999e-02]","[ 5.36785637e-06 5.39225630e-06 5.41665577e-06 ..., 8.52360651e-02 8.52409825e-02 8.52458999e-02]","[ 0.85138208 0.8509894 0.85064727 ..., 0.09263941 0.09265867 0.09267791]","[ 0.96578515 0.96544755 0.96510959 ..., 0.43874961 0.43874961 0.43874961]"
335.0,"[ 25. 25.10000038 25.20000076 ..., 899.79998779 899.90002441 900. ]",25.0,0.1,8751,"[ 0.00000000e+00 0.00000000e+00 0.00000000e+00 ..., 1.14710755e-10 1.13935743e-10 1.13166643e-10]",83.0,8.46158e-12,17.6,0.273973,"[ 5.95474005e-01 5.93567014e-01 5.91665983e-01 ..., 3.08021008e-05 3.06914008e-05 3.05811991e-05]","[ 5.95474005e-01 5.93567014e-01 5.91665983e-01 ..., 3.08021008e-05 3.06914008e-05 3.05811991e-05]","[ 0. 0. 0. ..., 0.18931125 0.18933147 0.18935168]","[ 0. 0. 0. ..., 0.18931125 0.18933147 0.18935168]","[ 0.85138208 0.8509894 0.85064727 ..., 0.09263941 0.09265867 0.09267791]","[ 0.96578515 0.96544755 0.96510959 ..., 0.43874961 0.43874961 0.43874961]"
171.0,"[ 25. 25.10000038 25.20000076 ..., 899.79998779 899.90002441 900. ]",25.0,0.1,8751,"[ 4.46234338e-08 4.47057964e-08 4.47890933e-08 ..., 2.74987561e-10 2.73075645e-10 2.71178691e-10]",83.0,8.46158e-12,17.7,0.273973,"[ 5.95474005e-01 5.93567014e-01 5.91665983e-01 ..., 3.08021008e-05 3.06914008e-05 3.05811991e-05]","[ 5.95474005e-01 5.93567014e-01 5.91665983e-01 ..., 3.08021008e-05 3.06914008e-05 3.05811991e-05]","[ 4.29414904e-05 4.31366789e-05 4.33318710e-05 ..., 2.93110162e-01 2.93112487e-01 2.93114781e-01]","[ 4.29414904e-05 4.31366789e-05 4.33318710e-05 ..., 2.93110162e-01 2.93112487e-01 2.93114781e-01]","[ 0.85138208 0.8509894 0.85064727 ..., 0.09263941 0.09265867 0.09267791]","[ 0.96578515 0.96544755 0.96510959 ..., 0.43874961 0.43874961 0.43874961]"
193.0,"[ 25. 25.10000038 25.20000076 ..., 899.79998779 899.90002441 900. ]",25.0,0.1,8751,"[ 0.00000000e+00 0.00000000e+00 0.00000000e+00 ..., 3.31152911e-10 3.28830518e-10 3.26526500e-10]",83.0,8.46158e-12,18.3,0.273973,"[ 5.95474005e-01 5.93567014e-01 5.91665983e-01 ..., 3.08021008e-05 3.06914008e-05 3.05811991e-05]","[ 5.95474005e-01 5.93567014e-01 5.91665983e-01 ..., 3.08021008e-05 3.06914008e-05 3.05811991e-05]","[ 0. 0. 0. ..., 0.32165378 0.32164654 0.32163933]","[ 0. 0. 0. ..., 0.32165378 0.32164654 0.32163933]","[ 0.85138208 0.8509894 0.85064727 ..., 0.09263941 0.09265867 0.09267791]","[ 0.96578515 0.96544755 0.96510959 ..., 0.43874961 0.43874961 0.43874961]"
211.0,"[ 25. 25.10000038 25.20000076 ..., 899.79998779 899.90002441 900. ]",25.0,0.1,8751,"[ 2.78900547e-09 2.79415358e-09 2.79935963e-09 ..., 2.51502985e-10 2.49762072e-10 2.48034815e-10]",83.0,8.46158e-12,18.3,0.273973,"[ 5.95474005e-01 5.93567014e-01 5.91665983e-01 ..., 3.08021008e-05 3.06914008e-05 3.05811991e-05]","[ 5.95474005e-01 5.93567014e-01 5.91665983e-01 ..., 3.08021008e-05 3.06914008e-05 3.05811991e-05]","[ 1.07354517e-05 1.07842488e-05 1.08330469e-05 ..., 2.80314744e-01 2.80321300e-01 2.80327857e-01]","[ 1.07354517e-05 1.07842488e-05 1.08330469e-05 ..., 2.80314744e